In [8]:
# ライブラリのインポート
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers, models
from keras.callbacks import TensorBoard, EarlyStopping
from keras.layers import Normalization

In [9]:
# DNNモデル
class DNNModel(models.Model):
    # 初期化
    def __init__(self, config):
        super(DNNModel, self).__init__()
        self.config = config
        self.normalizer = Normalization(axis=-1)
        self.dropout = layers.Dropout(config['dropout_rate'])
        self.dense_layers = [layers.Dense(units, activation="relu") for units in config['dense_units']]
        self.output_layer = layers.Dense(2)

    # 呼び出し
    def call(self, inputs):
        x = self.normalizer(inputs)                         # 訓練データの正規化
        for dense_layer in self.dense_layers:               # 全結合層
            x = dense_layer(x)
            x = self.dropout(x)
        return self.output_layer(x)                         # 出力層(2次元)

In [10]:
# 訓練
class DNNTrainer:
    # 初期化
    def __init__(self, config):
        self.config = config
        self.model = None

    # データの前処理
    def preprocess_data(self, dataset):
        data = dataset.iloc[:, list(range(4, 20))]
        label = dataset[['x', 'y']]
        return data.values, label.values

    # モデルのビルドとコンパイル
    def build_model(self, input_shape):
        self.model = DNNModel(self.config)
        self.model.build(input_shape)
        self.model.summary()
        
        # 学習率のスケジューリング
        lr_schedule = keras.optimizers.schedules.ExponentialDecay(
            self.config['initial_learning_rate'],
            decay_steps=self.config['decay_steps'],
            decay_rate=self.config['decay_rate'],
            staircase=True)

        optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)
        self.model.compile(optimizer=optimizer, loss="mean_squared_error", metrics=["accuracy"])

    # 訓練
    def train(self, train_dataset, validation_split=0.2):
        # 訓練データの前処理
        data, label = self.preprocess_data(train_dataset)
        
        # サマリーの表示
        if self.model is None:
            self.build_model(data.shape)
        else:
            self.model.summary()
        
        # 訓練データの正規化
        self.model.normalizer.adapt(data)
        
        # コールバック関数の定義
        callbacks = [
            EarlyStopping(monitor='val_loss', patience=self.config['patience']),
            TensorBoard(log_dir=self.config['log_dir'])
        ]

        # データの保存
        history = self.model.fit(
            data, label,
            validation_split=validation_split,
            batch_size=self.config['batch_size'],
            epochs=self.config['epochs'],
            callbacks=callbacks,
            verbose=1
        )
        
        self.model.save(self.config['model_path'])
        return history

    # 予測
    def predict(self, data):
        preprocessed_data, _ = self.preprocess_data(data)
        return self.model.predict(preprocessed_data)

In [11]:
# 設定
config = {
    'input_dim': 16,
    'dropout_rate': 0.1,
    'dense_units': [64, 32],
    'batch_size': 1024,
    'epochs': 1000,
    'patience': 40,
    'initial_learning_rate': 1e-3,
    'decay_steps': 10000,
    'decay_rate': 0.9,
    'log_dir': R'C:/Users/sukegawa/Desktop/study/logs/invfp16_1m',
    'model_path': R'C:/Users/sukegawa/Desktop/study/model/invfp16_1m'
}

# 訓練データの訓練
estimator = DNNTrainer(config)
train_dataset = pd.read_csv(R'C:/Users/sukegawa/Desktop/study/datasets/invfp/invfp16_1m.csv')
history = estimator.train(train_dataset)

# テストデータの評価
test_data = pd.read_csv(R'C:/Users/sukegawa/Desktop/study/datasets/invfp/invfp16_1m_test.csv')
predictions = estimator.predict(test_data)

Model: "dnn_model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_4 (Normalizat  multiple                 33        
 ion)                                                            
                                                                 
 dropout_4 (Dropout)         multiple                  0         
                                                                 
 dense_7 (Dense)             multiple                  1088      
                                                                 
 dense_8 (Dense)             multiple                  2080      
                                                                 
 dense_9 (Dense)             multiple                  66        
                                                                 
Total params: 3,267
Trainable params: 3,234
Non-trainable params: 33
____________________________________________________